# pytorch Model development

> Model development will be held here

In [3]:
#| default_exp pytorch_model_development

In [24]:
#| export
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.autograd import Variable
from torch.nn import functional as F
from sklearn.metrics import jaccard_score
from torch import nn
import numpy as np
import cv2
from torch.utils.data import Dataset
from typing import List, Tuple,Dict, Callable
import albumentations as A
import json
from fastcore.all import *

In [2]:
#| export 
class DoubleConv(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        dropout_rate=0.2):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
            )

    def forward(self, x):
        return self.conv(x)

In [3]:
#| export
class Pooling(nn.Module):
    def __init__(self, max_pool_only=False):
        super(Pooling, self).__init__()
        self.max_pool_only = max_pool_only
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        if self.max_pool_only:
            return self.max_pool(x)
        return torch.cat([self.max_pool(x), self.avg_pool(x)], axis=1)

In [4]:
#| export
class MaxAvgPool2d(nn.Module):
    def __init__(self, kernel_size, stride):
        super(MaxAvgPool2d, self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size, stride)
        self.avgpool = nn.AvgPool2d(kernel_size, stride)

    def forward(self, x):
        maxpooled = self.maxpool(x)
        avgpooled = self.avgpool(x)
        return torch.cat([maxpooled, avgpooled], 1)  # Concatenate along the channel dimension

In [5]:
#| export
class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

In [6]:
#| export
class UpConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpConv, self).__init__()
        self.up = nn.ConvTranspose2d(
                                     in_channels=in_channels, 
                                     out_channels=out_channels, 
                                     kernel_size=2, 
                                     stride=2)

    def forward(self, x, skip):
        x = self.up(x)
        x = torch.cat([x, skip], axis=1)
        return x

In [26]:
#| export
class UNet(nn.Module):
    def __init__(
                self, 
                in_channels:int=1, 
                out_channels:int=2,
                max_pool_only:bool=True,
                near_size:int=256,
                features:List[int]=[64, 128, 256, 512]
                ):
        super(UNet, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.size = near_size
        #if max_pool_only:
           #self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        #else:
           #self.pool = Pooling(max_pool_only=max_pool_only)
        self.pool = nn.MaxPool2d(
            kernel_size=2,
            stride=2,
        )

        # Down part of U-Net
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of U-Net
        for feature in reversed(features):
            self.ups.append(UpConv(feature*2, feature))
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def pad_image(self,image):
        _, _, h, w = image.shape
        if h != self.size or w!=self.size:
            pad_h = max(0, self.size -h)
            pad_w = max(0, self.size - w)
            image = nn.functional.pad(
                image, 
                (0, pad_w, 0, pad_h), 
                mode='constant', 
                value=0)
        return image, h, w

    def forward(self, x):

        x, h, w=self.pad_image(x)


        skip_connections = []

        # Down part of U-Net
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Up part of U-Net
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x, skip_connections[-idx//2-1])
            x = self.ups[idx+1](x)

        x = self.final_conv(x)
        return x[:,:, :h, :w]


In [27]:
#| hide
#pool=Pooling(max_pool_only=True)
#pool_o = pool(image)
#pool_o.shape

In [28]:
#| export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, size=None):
        super().__init__()
        self.size = size or (1, 1)
        self.ap = nn.AdaptiveAvgPool2d(self.size)
        self.mp = nn.AdaptiveMaxPool2d(self.size)

    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)


In [29]:
#| export
class AdaptiveAvgPool2d(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.output_size = output_size

    def forward(self, x):
        return F.adaptive_avg_pool2d(x, self.output_size)

class AdaptiveMaxPool2d(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.output_size = output_size

    def forward(self, x):
        return F.adaptive_max_pool2d(x, self.output_size)


In [30]:
#| hide
model = UNet(in_channels=1, out_channels=2,max_pool_only=False)
image = torch.randn(1, 1, 256, 256)
output = model(image)
output.shape


torch.Size([1, 2, 256, 256])

In [1]:
#| hide
import nbdev; nbdev.nbdev_export('08_pytorch_model_development.ipynb')